# 3-Stage Clos Network with Static VXLAN to make Networks Scalable
###### <sup>By Bhavani Parise, Deepti Chandra, Sarah Samuel; Guide format of use case: https://developer.cisco.com/docs/sonic/#!3-stage-clos-network-with-static-vxlan-to-make-networks-scalable</sup>


Modern cloud-scale data center networks require increased server-to-server communication over a network that stays resilient despite the rapid increase in the number of devices.

The 3-stage Clos network is a robust IP-BGP underlay network that enables the servers to communicate with each other with minimum latency. Over this network, you can configure overlay features such as Virtual Extensible Local Area Network (VXLAN).

A 3-stage Clos network consists of data center network switches where each spine switch connects to all leaf switches. And each leaf switch connects to a rack of servers in the data center. Any server in the data center is just three hops away from another server. The first hop is from the server to the directly connected leaf switch, the second hop is across the spine switches to the destination leaf switch, and the third hop is between the destination leaf switch to the destination server. This network architecture is highly scalable. Also, irrespective of the number of devices in the data center, the number of hops between the servers or the end-hosts is always 3, ensuring consistent latency in the data center network.

VXLAN is a tunneling protocol that stretches Layer 2 networks over an underlying Layer 3 IP network by encapsulating Layer 2 Ethernet frames within Layer 4 User Datagram Protocol (UDP). It then transports the encapsulated frames over a Layer 3 network.

You can create up to 16 million VXLANs instead of the traditional VLAN, which allows only 4096 VLANs in a network. Thus, VXLAN enables the building of highly scalable networks with physically distant Layer 2 network segments.

There are 2 types of VXLANS:

1. **Ethernet virtual private network (EVPN) VXLANs** - EVPN provides the control plane functionality for these VXLANs.

2. **Static VXLANs** - The router does not maintain a control plane for static VXLANs. So you should manually configure virtual tunnel end points and routes.

Though you use generally switches in a 3-stage Clos network, this notebook demonstrates how to [Bring Up 3-Stage Clos Network as an Underlay Network](#bring-up-3-stage-clos-network-as-an-underlay-network) using Cisco 8000 series routers that run SONiC. You can then [Configure Static VXLANs over the 3-Stage Clos Network](#configure-static-vxlans-over-the-3-stage-clos-network).

The following topology diagram depicts a simple 3-stage Clos network with two leaf routers in Tier-0 and two spine routers in Tier-1.

<center><img src="./images/522559.jpg" width="700"/></center>


This topology shows the static VXLAN overlay on the 3-stage Clos network:

<center><img src="./images/522560.jpg" width="700"/></center>


As you read through this notebook, play the code-cells, using the play button in the top left corner of this page, to send configuration commands to the live network nodes on the Cisco 8000 Emulator that runs in the background. The notebook refreshes the output of each cell-execution just beneath it.

After the topology is up, play through the following steps to [Bring Up 3 Stage Clos Network as an Underlay Network](#Bring-Up-3-Stage-Clos-Network-as-an-Underlay-Network):
* [Configure Host-Names](#Configure-Host-Names)
* [Assign IP-Addresses](#Assign-IP-Addresses)
* [Configure eBGP](#Configure-eBGP)
* [Verify BGP Route Exchange](#Verify-BGP-Route-Exchange)

After the 3-stage clos network is up, [Configure Static VXLANs over the 3-Stage Clos Network](#Configure-Static-VXLANs-over-the-3-Stage-Clos-Network), by playing through these steps:
* [Configure VLAN on Leaf Nodes](#Configure-VLAN-on-Leaf-Nodes)
* [Set Up and Apply Static VXLAN Configurations](#Set-Up-and-Apply-Static-VXLAN-Configurations)
* [Set Up and Load VNet Route Tables](#Set-Up-and-Load-VNet-Route-Tables)
* [Verify Static VXLAN](#Verify-Static-VXLAN)
* [Send Traffic across VXLAN Tunnel](#Send-Traffic-across-VXLAN-Tunnel)
* [Tested Static VXLAN Scale](#Tested-Static-VXLAN-Scale)

Finally, [clean up emulator session](#Clean-Up-Emulator-Session), once you are done.

### Bring up Topology

> Install prerequisite modules: This is required mainly for the first time you play this notebook. You can skip playing this cell for the subsequent runs.

In [1]:
%%capture cell-output
!sh lib/prereq_install.sh

> Play the following cell to bring up topology and access ssh console of each device in the topology. This cell takes about 10 - 15 minutes to complete execution. When you see the **Sim status** displayed as  **{'localhost': 'running'}** underneath this cell, the topology is up. Wait for a few more seconds until the code accesses the ssh console of each device. 

> Avoid playing this cell more than once, without [cleaning up the emulator session](#Clean-Up-Emulator-Session).

In [1]:
from lib.leaf_spine import *
nodes = {
         'S0':'', 
         'S1':'',
         'L0':'', 
         'L1':'', 
         'trex':''
        }
        
tb = access_device_consoles("lib/leaf_spine.yaml", nodes)

3.8.13 (default, Mar 28 2022, 11:38:47) 
[GCC 7.5.0]
Sim clean: Done
Simulation starting. Please wait for the Sim status message. This may take 10-15 minutes.
Sim status:  {'localhost': 'running'}
Consoles can be reached by:

 S0 : telnet 172.17.0.5 45843

 S1 : telnet 172.17.0.5 35050

 L0 : telnet 172.17.0.5 40745

 L1 : telnet 172.17.0.5 36819

 trex : telnet 172.17.0.5 42837

*** Logging into the devices ***

2022-08-04 16:54:50,478: %UNICON-INFO: +++ S0 logfile /tmp/S0-cli-20220804T165450477.log +++

2022-08-04 16:54:50,480: %UNICON-INFO: +++ Unicon plugin linux (unicon.plugins.linux) +++

2022-08-04 16:54:50,492: %UNICON-INFO: +++ connection to spawn: ssh -l cisco 172.17.0.5 -p 60113, id: 140668872884288 +++

2022-08-04 16:54:50,495: %UNICON-INFO: connection to S0
The authenticity of host '[172.17.0.5]:60113 ([172.17.0.5]:60113)' can't be established.
ECDSA key fingerprint is SHA256:BYsuXxg4AWOcokcLrkXUgzxbHBLv722mvYj+b03f2AE.
Are you sure you want to continue connecting (yes/no)

## Bring Up 3-Stage Clos Network as an Underlay Network

> To configure a 3-stage clos network, continue playing through the following steps:

### Configure Host-Names

> Configure host names for the spine and leaf routers for easy identification.

In [2]:
out = nodes['S0'].execute('sudo config hostname SPINE0')
out = nodes['S0'].execute('sudo config save -y')
out = nodes['S1'].execute('sudo config hostname SPINE1')
out = nodes['S1'].execute('sudo config save -y')
out = nodes['L0'].execute('sudo config hostname LEAF0')
out = nodes['L0'].execute('sudo config save -y')
out = nodes['L1'].execute('sudo config hostname LEAF1')
out = nodes['L1'].execute('sudo config save -y')


2022-08-04 17:01:48,168: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config hostname SPINE0' +++
sudo config hostname SPINE0
Running command: service hostname-config restart
Reloading Monit configuration ...
Reinitializing monit daemon
Please note loaded setting will be lost after system reboot. To preserve setting, run `config save`.
cisco@sonic:~$ 

2022-08-04 17:01:49,554: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config save -y' +++
sudo config save -y
Running command: /usr/local/bin/sonic-cfggen -d --print-data > /etc/sonic/config_db.json
cisco@sonic:~$ 

2022-08-04 17:01:50,791: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config hostname SPINE1' +++
sudo config hostname SPINE1
Running command: service hostname-config restart
Reloading Monit configuration ...
Reinitializing monit daemon
Please note loaded setting will be lost after system reboot. To preserve setting, run `config save`.
cisco@sonic:~$ 

2022-08-04 17:01

### Assign IP Addresses

> This step assigns IP addresses on the interfaces of all devices as per the topology diagram. And then saves the configurations.

In [3]:
out = nodes['S0'].execute('sudo config interface ip add Ethernet0 10.0.1.1/24')
out = nodes['S0'].execute('sudo config interface ip add Ethernet2 10.0.2.1/24')
out = nodes['S0'].execute('sudo config interface ip add Loopback0 10.10.10.100/32')
out = nodes['S0'].execute('sudo config save -y')
out = nodes['S1'].execute('sudo config interface ip add Ethernet0 10.0.3.1/24')
out = nodes['S1'].execute('sudo config interface ip add Ethernet2 10.0.4.1/24')
out = nodes['S1'].execute('sudo config interface ip add Loopback0 10.10.11.100/32')
out = nodes['S1'].execute('sudo config save -y')
out = nodes['L0'].execute('sudo config interface ip add Ethernet0 10.0.1.2/24')
out = nodes['L0'].execute('sudo config interface ip add Ethernet3 10.0.3.2/24')
out = nodes['L0'].execute('sudo config interface ip add Ethernet2 10.0.5.1/24')
out = nodes['L0'].execute('sudo config interface ip add Loopback0 10.10.10.200/32')
out = nodes['L0'].execute('sudo config save -y')
out = nodes['L1'].execute('sudo config interface ip add Ethernet3 10.0.2.2/24')
out = nodes['L1'].execute('sudo config interface ip add Ethernet0 10.0.4.2/24')
out = nodes['L1'].execute('sudo config interface ip add Ethernet2 10.0.6.1/24')
out = nodes['L1'].execute('sudo config interface ip add Loopback0 10.10.11.200/32')
out = nodes['L1'].execute('sudo config save -y')
out = nodes['trex'].execute('ifconfig eth1 10.0.5.2 netmask 255.255.255.0 up')
out = nodes['trex'].execute('ifconfig eth2 10.0.6.2 netmask 255.255.255.0 up')


2022-08-04 17:02:02,281: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config interface ip add Ethernet0 10.0.1.1/24' +++
sudo config interface ip add Ethernet0 10.0.1.1/24
cisco@sonic:~$ 

2022-08-04 17:02:03,170: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config interface ip add Ethernet2 10.0.2.1/24' +++
sudo config interface ip add Ethernet2 10.0.2.1/24
cisco@sonic:~$ 

2022-08-04 17:02:03,947: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config interface ip add Loopback0 10.10.10.100/32' +++
sudo config interface ip add Loopback0 10.10.10.100/32
cisco@sonic:~$ 

2022-08-04 17:02:04,839: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config save -y' +++
sudo config save -y
Running command: /usr/local/bin/sonic-cfggen -d --print-data > /etc/sonic/config_db.json
cisco@sonic:~$ 

2022-08-04 17:02:06,096: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config interface ip add Ethernet0 10.0.3.1/

> Verify the configured IP addresses using the ```show ip interfaces``` command on the routers and ```ifconfig -a``` command on the TREX server.

In [4]:
for n in nodes:
   if (n == 'trex'):
      out = nodes[n].execute('ifconfig -a eth1')
      out = nodes[n].execute('ifconfig -a eth2')
   else:
      out = nodes[n].execute('show ip interfaces')  


2022-08-04 17:02:44,032: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show ip interfaces' +++
show ip interfaces
Interface    Master    IPv4 address/mask    Admin/Oper    BGP Neighbor    Neighbor IP
-----------  --------  -------------------  ------------  --------------  -------------
Ethernet0              10.0.1.1/24          up/up         N/A             N/A
Ethernet2              10.0.2.1/24          up/up         N/A             N/A
Loopback0              10.10.10.100/32      up/up         N/A             N/A
docker0                240.127.1.1/24       up/down       N/A             N/A
eth0                   192.168.122.250/24   up/up         N/A             N/A
lo                     127.0.0.1/16         up/up         N/A             N/A
cisco@sonic:~$ 

2022-08-04 17:02:45,559: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show ip interfaces' +++
show ip interfaces
Interface    Master    IPv4 address/mask    Admin/Oper    BGP Neighbor    Neighbor IP


### Configure eBGP

> The following section shows the eBGP (exterior Border Gateway Protocol) configuration on all spine and leaf routers. The spines are in Autonomous System (AS) 100 and the leaves in AS 200. 
> Note the usage of the ```allowas-in``` option in the ```neighbor``` command. The default behaviour of BGP is to reject learning routes from own AS. The ```allowas-in``` option overrides this default behavior and the nodes learn routes from own AS. 

In [5]:
out = nodes['S0'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname SPINE_0' \
-c 'router-id 10.10.10.100' \
-c 'router bgp 100' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.2.2 remote-as 200' \
-c 'neighbor 10.0.1.2 remote-as 200' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.2.2 allowas-in' \
-c 'neighbor 10.0.1.2 allowas-in' \
-c 'network 10.0.1.0/24' \
-c 'network 10.0.2.0/24' \
-c 'network 10.10.10.100/32' \
-c 'redistribute connected'
''')
out = nodes['S1'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname SPINE_1' \
-c 'router-id 10.10.11.100' \
-c 'router bgp 100' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.3.2 remote-as 200' \
-c 'neighbor 10.0.4.2 remote-as 200' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.3.2 allowas-in' \
-c 'neighbor 10.0.4.2 allowas-in' \
-c 'network 10.0.3.0/24' \
-c 'network 10.0.4.0/24' \
-c 'network 10.10.11.100/32' \
-c 'redistribute connected'
''')
out = nodes['L0'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname LEAF_0' \
-c 'router-id 10.10.10.200' \
-c 'router bgp 200' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.1.1 remote-as 100' \
-c 'neighbor 10.0.3.1 remote-as 100' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.1.1 allowas-in' \
-c 'neighbor 10.0.3.1 allowas-in' \
-c 'network 10.0.1.0/24' \
-c 'network 10.0.3.0/24' \
-c 'network 10.0.5.0/24' \
-c 'network 10.10.10.200/32' \
-c 'redistribute connected'
''')
out = nodes['L1'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname LEAF_1' \
-c 'router-id 10.10.11.200' \
-c 'router bgp 200' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.2.1 remote-as 100' \
-c 'neighbor 10.0.4.1 remote-as 100' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.2.1 allowas-in' \
-c 'neighbor 10.0.4.1 allowas-in' \
-c 'network 10.0.2.0/24' \
-c 'network 10.0.4.0/24' \
-c 'network 10.0.6.0/24' \
-c 'network 10.10.11.200/32' \
-c 'redistribute connected'
''')


2022-08-04 17:02:54,677: %UNICON-INFO: +++ sonic with via 'cli': executing command 'vtysh -c 'configure terminal' -c 'hostname SPINE_0' -c 'router-id 10.10.10.100' -c 'router bgp 100' -c 'no bgp ebgp-requires-policy' -c 'neighbor 10.0.2.2 remote-as 200' -c 'neighbor 10.0.1.2 remote-as 200' -c 'address-family ipv4 unicast' -c 'neighbor 10.0.2.2 allowas-in' -c 'neighbor 10.0.1.2 allowas-in' -c 'network 10.0.1.0/24' -c 'network 10.0.2.0/24' -c 'network 10.10.10.100/32' -c 'redistribute connected'' +++
' -c 'redistribute connected'ress-family ipv4 unicast' -c 'neighbor 10.0.2.2 allowas-in' -c 'neighbor 10.0.1.2 allowas-in' -c 'network 10.0.1.0/24' -c 'network 10.0.2.0/24' -c 'network 10.10.10.100/32'
cisco@sonic:~$ 

2022-08-04 17:02:55,332: %UNICON-INFO: +++ sonic with via 'cli': executing command 'vtysh -c 'configure terminal' -c 'hostname SPINE_1' -c 'router-id 10.10.11.100' -c 'router bgp 100' -c 'no bgp ebgp-requires-policy' -c 'neighbor 10.0.3.2 remote-as 200' -c 'neighbor 10.0.4.2 

### Verify BGP Route Exchange

> Execute the command ```show ip bgp``` on all tier-0 and tier-1 routers to ensure eBGP is up.

In [6]:
for n in nodes:
   if (n != 'trex'):
      out = nodes[n].execute('''vtysh \
      -c 'show ip bgp'
      ''')


2022-08-04 17:02:59,807: %UNICON-INFO: +++ sonic with via 'cli': executing command 'vtysh       -c 'show ip bgp'' +++
vtysh       -c 'show ip bgp'
BGP table version is 13, local router ID is 10.10.10.100, vrf id 0
Default local pref 100, local AS 100
Status codes:  s suppressed, d damped, h history, * valid, > best, = multipath,
               i internal, r RIB-failure, S Stale, R Removed
Nexthop codes: @NNN nexthop's vrf id, < announce-nh-self
Origin codes:  i - IGP, e - EGP, ? - incomplete

   Network          Next Hop            Metric LocPrf Weight Path
*  10.0.1.0/24      10.0.2.2                               0 200 100 i
*                   10.0.1.2                 0             0 200 i
*                   0.0.0.0                  0         32768 ?
*>                  0.0.0.0                  0         32768 i
*  10.0.2.0/24      10.0.2.2                 0             0 200 i
*                   10.0.1.2                               0 200 100 i
*                   0.0.0.0      

> This section checks the routes learnt through BGP by executing the command ```show ip route``` on all nodes in tier-0 and tier-1. The lines starting with the letter **B** are the routes learnt through BGP.

In [7]:
for n in nodes:
   if (n != 'trex'):
      out = nodes[n].execute('''vtysh \
      -c 'show ip route'
      ''')


2022-08-04 17:03:06,428: %UNICON-INFO: +++ sonic with via 'cli': executing command 'vtysh       -c 'show ip route'' +++
vtysh       -c 'show ip route'
Codes: K - kernel route, C - connected, S - static, R - RIP,
       O - OSPF, I - IS-IS, B - BGP, E - EIGRP, N - NHRP,
       T - Table, v - VNC, V - VNC-Direct, A - Babel, D - SHARP,
       F - PBR, f - OpenFabric,
       > - selected route, * - FIB route, q - queued, r - rejected, b - backup

K>* 0.0.0.0/0 [0/202] via 192.168.122.1, eth0, 00:09:46
C>* 10.0.1.0/24 is directly connected, Ethernet0, 00:01:04
C>* 10.0.2.0/24 is directly connected, Ethernet2, 00:01:03
B>* 10.0.3.0/24 [20/0] via 10.0.1.2, Ethernet0, weight 1, 00:00:09
B>* 10.0.4.0/24 [20/0] via 10.0.2.2, Ethernet2, weight 1, 00:00:08
B>* 10.0.5.0/24 [20/0] via 10.0.1.2, Ethernet0, weight 1, 00:00:09
B>* 10.0.6.0/24 [20/0] via 10.0.2.2, Ethernet2, weight 1, 00:00:08
C>* 10.10.10.100/32 is directly connected, Loopback0, 00:01:02
B>* 10.10.10.200/32 [20/0] via 10.0.1.2, Ethern

> Ping IP addresses on the LEAF0 (L0) router from the LEAF1 (L1) router and vice-versa. This ensures that the 3-stage Clos network is ready for end-to-end traffic flow.

In [8]:
out = nodes['L1'].execute('ping -c5 10.0.5.1')
out = nodes['L1'].execute('ping -c5 10.10.10.200')
out = nodes['L0'].execute('ping -c5 10.0.6.1')
out = nodes['L0'].execute('ping -c5 10.10.11.200')


2022-08-04 17:03:13,007: %UNICON-INFO: +++ sonic with via 'cli': executing command 'ping -c5 10.0.5.1' +++
ping -c5 10.0.5.1
PING 10.0.5.1 (10.0.5.1) 56(84) bytes of data.
64 bytes from 10.0.5.1: icmp_seq=1 ttl=63 time=37.0 ms
64 bytes from 10.0.5.1: icmp_seq=2 ttl=63 time=25.7 ms
64 bytes from 10.0.5.1: icmp_seq=3 ttl=63 time=21.4 ms
64 bytes from 10.0.5.1: icmp_seq=4 ttl=63 time=31.9 ms
64 bytes from 10.0.5.1: icmp_seq=5 ttl=63 time=53.9 ms

--- 10.0.5.1 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 10ms
rtt min/avg/max/mdev = 21.362/33.974/53.870/11.287 ms
cisco@sonic:~$ 

2022-08-04 17:03:17,221: %UNICON-INFO: +++ sonic with via 'cli': executing command 'ping -c5 10.10.10.200' +++
ping -c5 10.10.10.200
PING 10.10.10.200 (10.10.10.200) 56(84) bytes of data.
64 bytes from 10.10.10.200: icmp_seq=1 ttl=63 time=88.8 ms
64 bytes from 10.10.10.200: icmp_seq=2 ttl=63 time=23.7 ms
64 bytes from 10.10.10.200: icmp_seq=3 ttl=63 time=154 ms
64 bytes from 10.10.10

**Pinging to remote leaf ip address is successful and so the 3-stage Clos network is up and ready for data traffic**

> You have now successfully brought up a simple 3-stage Clos network, sent traffic across it. This forms the IP-BGP underlay for your data center network. 

> Scale up these configurations as per the number of devices in your data centre. 

> The next step is to configure an overlay. To configure static VXLAN overlay over the 3-stage Clos network, continue to the next section.


## Configure Static VXLANs over the 3-Stage Clos Network 

> In this section , the notebook demostrates how to configure static VXLAN tunnels between LEAF0 and LEAF1. The underlay consists of the 3-stage Clos network which we configured in the previous section. 


### Configure VLAN on Leaf Nodes

> Configure VLAN 10 on both LEAF routers to set up VXLAN Tunnels between the VLAN segments on the LEAFs. Then remove the IP addresses assigned to the LEAF interfaces connected to the traffic generator and assign the VLAN 10. Assign the IP address that we removed to the VLAN interface.

In [37]:
out = nodes['L0'].execute('sudo config vlan add 10')
out = nodes['L0'].execute('sudo config interface ip remove Ethernet2 10.0.5.1/24')
out = nodes['L0'].execute('sudo config vlan member add -u 10 Ethernet2')
out = nodes['L0'].execute('sudo config interface ip add Vlan10 10.0.5.1/24')
out = nodes['L0'].execute('show vlan brief')

out = nodes['L1'].execute('sudo config vlan add 10')
out = nodes['L1'].execute('sudo config interface ip remove Ethernet2 10.0.6.1/24')
out = nodes['L1'].execute('sudo config vlan member add -u 10 Ethernet2')
out = nodes['L1'].execute('sudo config interface ip add Vlan10 10.0.6.1/24')
out = nodes['L1'].execute('show vlan brief')


2022-08-04 13:37:02,066: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config vlan add 10' +++
sudo config vlan add 10
cisco@sonic:~$ 

2022-08-04 13:37:02,888: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config interface ip remove Ethernet2 10.0.5.1/24' +++
sudo config interface ip remove Ethernet2 10.0.5.1/24
cisco@sonic:~$ 

2022-08-04 13:37:03,768: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config vlan member add -u 10 Ethernet2' +++
sudo config vlan member add -u 10 Ethernet2
cisco@sonic:~$ 

2022-08-04 13:37:04,583: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config interface ip add Vlan10 10.0.5.1/24' +++
sudo config interface ip add Vlan10 10.0.5.1/24
cisco@sonic:~$ 

2022-08-04 13:37:05,360: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show vlan brief' +++
show vlan brief
+-----------+--------------+-----------+----------------+-----------------------+-------------+
|   VLAN ID | IP A

### Set Up and Apply Static VXLAN Configurations

> Static VXLAN configurations are applied using JSON files. This step creates the files with the VXLAN configurations and loads them on LEAF0 and LEAF1. It configures VXLAN with a Virtual Network Identifier (VNI) of 1000 on LEAF0. The source IP address of the tunnel is the IP address of the Loopback 0 interface of LEAF0. This VXLAN maps VLAN 10 with a VNI of 1000.

In [38]:
# This cell creates the json file
import json
  
# VXLAN Configurations
VXLAN_dictionary ={
    "VXLAN_TUNNEL": {
        "tunnel_v4": {
            "src_ip": "10.10.10.200"
        }
    },
    "VNET": {
        "Vnet_1000": {
            "vxlan_tunnel": "tunnel_v4",
            "vni": "1000",
            "scope": "default"
        }
    },
    "VLAN_INTERFACE": {
        "Vlan10": {
            "vnet_name": "Vnet_1000",
            "vni": "1000"
        },
         "Vlan10|10.0.5.1/24": {}
    }
}
    
json_object = json.dumps(VXLAN_dictionary, indent = 4)
# Writing to json file
with open("VXLAN.json", "w") as outfile:
    outfile.write(json_object)

# Copying json file to LEAF0    
rtr_ip = str(nodes['L0'].connections.cli.ip)
rtr_port = str(nodes['L0'].connections.cli.port)
src_file = "./VXLAN.json"
dst_on_rtr = "/home/cisco/VXLAN.json"
copy_file_to_rtr(rtr_ip, rtr_port, src_file, dst_on_rtr)

# Check if the file is copied on LEAF0
out = nodes['L0'].execute('cat VXLAN.json')

# Apply and save the VXLAN configurations on LEAF0
out = nodes['L0'].execute('sudo config load VXLAN.json -y')
out = nodes['L0'].execute('sudo config save -y')

# Copy file to swss container
out = nodes['L0'].execute('docker cp VXLAN.json swss://.')

# Load configs in the swss container
out = nodes['L0'].execute('docker exec -i swss swssconfig VXLAN.json')


2022-08-04 13:37:14,809: %UNICON-INFO: +++ sonic with via 'cli': executing command 'cat VXLAN.json' +++
cat VXLAN.json
{
    "VXLAN_TUNNEL": {
        "tunnel_v4": {
            "src_ip": "10.10.10.200"
        }
    },
    "VNET": {
        "Vnet_1000": {
            "vxlan_tunnel": "tunnel_v4",
            "vni": "1000",
            "scope": "default"
        }
    },
    "VLAN_INTERFACE": {
        "Vlan10": {
            "vnet_name": "Vnet_1000",
            "vni": "1000"
        },
        "Vlan10|10.0.5.1/24": {}
    }
}cisco@sonic:~$ 

2022-08-04 13:37:14,992: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config load VXLAN.json -y' +++
sudo config load VXLAN.json -y
Running command: /usr/local/bin/sonic-cfggen -j VXLAN.json --write-to-db
cisco@sonic:~$ 

2022-08-04 13:37:16,082: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config save -y' +++
sudo config save -y
Running command: /usr/local/bin/sonic-cfggen -d --print-data > /etc/sonic/config

> Like LEAF0, on LEAF1 configure VXLAN with a VNI of 1000, which maps to VLAN 10. The source IP address of the VXLAN tunnel is the IP address of the Loopback0 interface of LEAF1.

In [39]:
# This cell creates the json file  
# VXLAN Configurations
VXLAN_dictionary ={
    "VXLAN_TUNNEL": {
        "tunnel_v4": {
            "src_ip": "10.10.11.200"
        }
    },
    "VNET": {
        "Vnet_1000": {
            "vxlan_tunnel": "tunnel_v4",
            "vni": "1000",
            "scope": "default"
        }
    },
    "VLAN_INTERFACE": {
        "Vlan10": {
            "vnet_name": "Vnet_1000",
            "vni": "1000"
        },
         "Vlan10|10.0.6.1/24": {}
    }
}
json_object = json.dumps(VXLAN_dictionary, indent = 4)
  
# Writing to json file
with open("VXLAN2.json", "w") as outfile:
    outfile.write(json_object)

# Copying json file to LEAF1
rtr_ip = str(nodes['L1'].connections.cli.ip)
rtr_port = str(nodes['L1'].connections.cli.port)
src_file = "./VXLAN2.json"
dst_on_rtr = "/home/cisco/VXLAN2.json"
copy_file_to_rtr(rtr_ip, rtr_port, src_file, dst_on_rtr)

# Check if the file is copied on LEAF1
out = nodes['L1'].execute('cat VXLAN2.json')

# Apply and save the VXLAN configurations on LEAF0
out = nodes['L1'].execute('sudo config load VXLAN2.json -y')
out = nodes['L1'].execute('sudo config save -y')

# Copy file to swss container
out = nodes['L1'].execute('docker cp VXLAN2.json swss://.')

# Load configs in the swss container
out = nodes['L1'].execute('docker exec -i swss swssconfig VXLAN2.json')


2022-08-04 13:37:25,001: %UNICON-INFO: +++ sonic with via 'cli': executing command 'cat VXLAN2.json' +++
cat VXLAN2.json
{
    "VXLAN_TUNNEL": {
        "tunnel_v4": {
            "src_ip": "10.10.11.200"
        }
    },
    "VNET": {
        "Vnet_1000": {
            "vxlan_tunnel": "tunnel_v4",
            "vni": "1000",
            "scope": "default"
        }
    },
    "VLAN_INTERFACE": {
        "Vlan10": {
            "vnet_name": "Vnet_1000",
            "vni": "1000"
        },
        "Vlan10|10.0.6.1/24": {}
    }
}cisco@sonic:~$ 

2022-08-04 13:37:25,191: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config load VXLAN2.json -y' +++
sudo config load VXLAN2.json -y
Running command: /usr/local/bin/sonic-cfggen -j VXLAN2.json --write-to-db
cisco@sonic:~$ 

2022-08-04 13:37:26,170: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config save -y' +++
sudo config save -y
Running command: /usr/local/bin/sonic-cfggen -d --print-data > /etc/sonic/c

### Set Up and Load VNet Route Tables

> Since static VXLAN does not have a control plane to learn routes, set up the route tables manually by playing the below cell, SONiC uses the SWitch State Service (SWSS) docker container to maintain the database of routes. This cell sets up the JSON file for the Virtual Network (VNet) route table and copies it to the SWSS container using the ```docker cp``` command. The tables are then loaded in the SWSS container using the ```docker exec -i swss swssconfig``` command.

In [40]:
# Setting up the VNET table with routes
VNET_ROUTE =[{ "VNET_ROUTE_TUNNEL_TABLE:Vnet_1000:10.0.6.0/24": {"endpoint": "10.10.11.200"},"OP": "SET"},{ "VNET_ROUTE_TUNNEL_TABLE:Vnet_1000:10.0.5.2/32": {"ifname": "Vlan10"},"OP": "SET"}]

# Writing to json file
json_object2 = json.dumps(VNET_ROUTE, indent = 4)
with open("VNET_ROUTE.json", "w") as outfile:
    outfile.write(json_object2)
    
# Copying json file to LEAF0
rtr_ip = str(nodes['L0'].connections.cli.ip)
rtr_port = str(nodes['L0'].connections.cli.port)
src_file = "./VNET_ROUTE.json"
dst_on_rtr = "/home/cisco/vnt.route_snhop.json"
copy_file_to_rtr(rtr_ip, rtr_port, src_file, dst_on_rtr)

# Check if the file with route table is copied on LEAF0
out = nodes['L0'].execute('cat vnt.route_snhop.json')

# Copy file with the route table over to swss container
out = nodes['L0'].execute('docker cp vnt.route_snhop.json swss://.')

# Load the routes in the swss container
out = nodes['L0'].execute('docker exec -i swss swssconfig vnt.route_snhop.json')


2022-08-04 13:37:33,286: %UNICON-INFO: +++ sonic with via 'cli': executing command 'cat vnt.route_snhop.json' +++
cat vnt.route_snhop.json
[
    {
        "VNET_ROUTE_TUNNEL_TABLE:Vnet_1000:10.0.6.0/24": {
            "endpoint": "10.10.11.200"
        },
        "OP": "SET"
    },
    {
        "VNET_ROUTE_TUNNEL_TABLE:Vnet_1000:10.0.5.2/32": {
            "ifname": "Vlan10"
        },
        "OP": "SET"
    }
]cisco@sonic:~$ 

2022-08-04 13:37:33,482: %UNICON-INFO: +++ sonic with via 'cli': executing command 'docker cp vnt.route_snhop.json swss://.' +++
docker cp vnt.route_snhop.json swss://.
cisco@sonic:~$ 

2022-08-04 13:37:34,592: %UNICON-INFO: +++ sonic with via 'cli': executing command 'docker exec -i swss swssconfig vnt.route_snhop.json' +++
docker exec -i swss swssconfig vnt.route_snhop.json
cisco@sonic:~$ 


> Set up the route table on LEAF1

In [41]:
# Setting up the VNET table with routes

VNET_ROUTE =[{ "VNET_ROUTE_TUNNEL_TABLE:Vnet_1000:10.0.5.0/24": {"endpoint": "10.10.10.200"},"OP": "SET"},{ "VNET_ROUTE_TUNNEL_TABLE:Vnet_1000:10.0.6.2/32": {"ifname": "Vlan10"},"OP": "SET"}]

# Writing to json file
json_object2 = json.dumps(VNET_ROUTE, indent = 4)
with open("VNET_ROUTE2.json", "w") as outfile:
    outfile.write(json_object2)
    
# Copying json file to LEAF1
rtr_ip = str(nodes['L1'].connections.cli.ip)
rtr_port = str(nodes['L1'].connections.cli.port)   
src_file = "./VNET_ROUTE2.json"
dst_on_rtr = "/home/cisco/vnt.route_snhop.json"
copy_file_to_rtr(rtr_ip, rtr_port, src_file, dst_on_rtr)

# Check if the file is copied on LEAF1
out = nodes['L1'].execute('cat vnt.route_snhop.json')

# Copy file with the route table over to swss container
out = nodes['L1'].execute('docker cp vnt.route_snhop.json swss://.')

# Load the routes in the swss container
out = nodes['L1'].execute('docker exec -i swss swssconfig vnt.route_snhop.json')


2022-08-04 13:37:39,453: %UNICON-INFO: +++ sonic with via 'cli': executing command 'cat vnt.route_snhop.json' +++
cat vnt.route_snhop.json
[
    {
        "VNET_ROUTE_TUNNEL_TABLE:Vnet_1000:10.0.5.0/24": {
            "endpoint": "10.10.10.200"
        },
        "OP": "SET"
    },
    {
        "VNET_ROUTE_TUNNEL_TABLE:Vnet_1000:10.0.6.2/32": {
            "ifname": "Vlan10"
        },
        "OP": "SET"
    }
]cisco@sonic:~$ 

2022-08-04 13:37:39,644: %UNICON-INFO: +++ sonic with via 'cli': executing command 'docker cp vnt.route_snhop.json swss://.' +++
docker cp vnt.route_snhop.json swss://.
cisco@sonic:~$ 

2022-08-04 13:37:40,584: %UNICON-INFO: +++ sonic with via 'cli': executing command 'docker exec -i swss swssconfig vnt.route_snhop.json' +++
docker exec -i swss swssconfig vnt.route_snhop.json
cisco@sonic:~$ 


### Verify Static VXLAN

> Execute show commands on LEAF0 router to check the VXLAN configurations.

In [42]:
out = nodes['L0'].execute('show vxlan interface')
out = nodes['L0'].execute('show vnet brief')
out = nodes['L0'].execute('show vnet name Vnet_1000')
out = nodes['L0'].execute('show vnet route all')
out = nodes['L0'].execute('show ip route')


2022-08-04 13:37:47,304: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show vxlan interface' +++
show vxlan interface
VTEP Information:

	VTEP Name : tunnel_v4, SIP  : 10.10.10.200
	Source interface  : Loopback0
cisco@sonic:~$ 

2022-08-04 13:37:48,256: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show vnet brief' +++
show vnet brief
vnet name    vxlan tunnel      vni  peer list
-----------  --------------  -----  -----------
Vnet_1000    tunnel_v4        1000
cisco@sonic:~$ 

2022-08-04 13:37:48,983: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show vnet name Vnet_1000' +++
show vnet name Vnet_1000
vnet name    vxlan tunnel      vni  peer list
-----------  --------------  -----  -----------
Vnet_1000    tunnel_v4        1000
cisco@sonic:~$ 

2022-08-04 13:37:49,809: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show vnet route all' +++
show vnet route all
vnet name    prefix    nexthop    interface
-----------  --------  ---------

> Execute show commands on LEAF1 router to check the VXLAN configurations.

In [43]:
out = nodes['L1'].execute('show vxlan interface')
out = nodes['L1'].execute('show vnet brief')
out = nodes['L1'].execute('show vnet name Vnet_1000')
out = nodes['L1'].execute('show vnet route all')
out = nodes['L1'].execute('show ip route')


2022-08-04 13:37:57,118: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show vxlan interface' +++
show vxlan interface
VTEP Information:

	VTEP Name : tunnel_v4, SIP  : 10.10.11.200
	Source interface  : Loopback0
cisco@sonic:~$ 

2022-08-04 13:37:57,983: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show vnet brief' +++
show vnet brief
vnet name    vxlan tunnel      vni  peer list
-----------  --------------  -----  -----------
Vnet_1000    tunnel_v4        1000
cisco@sonic:~$ 

2022-08-04 13:37:58,832: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show vnet name Vnet_1000' +++
show vnet name Vnet_1000
vnet name    vxlan tunnel      vni  peer list
-----------  --------------  -----  -----------
Vnet_1000    tunnel_v4        1000
cisco@sonic:~$ 

2022-08-04 13:37:59,591: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show vnet route all' +++
show vnet route all
vnet name    prefix    nexthop    interface
-----------  --------  ---------

In [48]:
out = nodes['L0'].execute('ping -c5 10.0.5.2')
out = nodes['L1'].execute('ping -c5 10.0.6.2')


2022-08-04 13:41:50,348: %UNICON-INFO: +++ sonic with via 'cli': executing command 'ping -c5 10.0.5.2' +++
ping -c5 10.0.5.2
PING 10.0.5.2 (10.0.5.2) 56(84) bytes of data.

--- 10.0.5.2 ping statistics ---
5 packets transmitted, 0 received, 100% packet loss, time 104ms

cisco@sonic:~$ 

2022-08-04 13:42:04,603: %UNICON-INFO: +++ sonic with via 'cli': executing command 'ping -c5 10.0.6.2' +++
ping -c5 10.0.6.2
PING 10.0.6.2 (10.0.6.2) 56(84) bytes of data.

--- 10.0.6.2 ping statistics ---
5 packets transmitted, 0 received, 100% packet loss, time 79ms

cisco@sonic:~$ 


### Send Traffic across VXLAN Tunnel

> To simulate server-to-server traffic flow across VXLAN tunnel, connect the TREX software traffic generator ports as the end-hosts to LEAF0 and LEAF1. 

> Details of traffic stream injected into LEAF0 from TREX:
> * Source IP address: 10.0.5.2
> * Destination IP address: 10.0.6.2

> Details of traffic stream injected into LEAF1 from TREX:
> * Source IP address: 10.0.6.2
> * Destination IP address: 10.0.5.2

<center><img src="./images/vxlan-traffic.png" width="700"/></center>

> The function ```generate_bidir_traffic``` injects a bidirectional traffic burst for 1 second. After the cell execution, check ```Total-tx-pkt``` and ```Total-rx-pkt``` in the ```summary stats``` at the end of the output to ensure that there is no traffic loss.

In [47]:
trex_ipaddress = str(nodes['trex'].connections.cli.ip)
trex_port = str(nodes['trex'].connections.cli.port)

generate_bidir_traffic(trex_ipaddress, trex_port)

Last login: Thu Aug  4 13:29:40 2022 from gateway
[root@localhost ~]# ifconfig eth1 up; ifconfig eth2 up
[root@localhost ~]# cd /opt/cisco/trex/latest/cap2/; ls -lart test-new.yaml; cd  /opt/cisco/trex/latest/
-rw-r--r-- 1 root root 557 Aug  4 13:41 test-new.yaml
[root@localhost latest]# ./t-rex-64 -f cap2/test-new.yaml -m 300 -d 1
exit
Trying to bind to vfio-pci ...
Trying to compile and bind to igb_uio ...
ERROR: We don't have precompiled igb_uio.ko module for your kernel version.
Will try compiling automatically...
Success.

/usr/bin/python3 dpdk_nic_bind.py --bind=igb_uio 0000:00:04.0 0000:00:05.0 
The ports are bound/configured.
Starting  TRex v2.99 please wait  ... 
 set driver name net_virtio 
 driver capability  : SLRO 
 set dpdk queues mode to ONE_QUE 
 Number of ports found: 2
zmq publisher at: tcp://*:4500
 wait 1 sec .
port : 0 
------------
link         :  link : Link Up - speed 100000 Mbps - full-duplex
promiscuous  : 0 
port : 1 
------------
link         :  link : Link 

> Check the summary stats in the trex output above, for packet drops. And then verify the MAC addresses learnt on the leaf routers, L0 and L1.

In [26]:
out = nodes['L0'].execute('show mac')
out = nodes['L1'].execute('show mac')


2022-08-04 06:42:12,246: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show mac' +++
show mac
  No.    Vlan  MacAddress         Port       Type
-----  ------  -----------------  ---------  -------
    1      10  02:11:39:9B:F7:7D  Ethernet2  Dynamic
Total number of entries 1
cisco@sonic:~$ 

2022-08-04 06:42:13,108: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show mac' +++
show mac
  No.    Vlan  MacAddress         Port       Type
-----  ------  -----------------  ---------  -------
    1      10  02:62:9A:25:90:1D  Ethernet2  Dynamic
Total number of entries 1
cisco@sonic:~$ 


> You have now successfully brought up a VXLAN Tunnel over a 3-stage Clos network, sent traffic across the VXLAN. 

### Tested Static VXLAN Scale

> A scale of 32K remote Virtual Tunnel End Points (VTEP) with 128K unique overlay-prefix and VTEP Encap entries has been demonstrated on Cisco 8000 routers with SONiC. For more details of this demo by the Cisco team, refer https://blogs.cisco.com/sp/cisco-and-sonic

### Clean up Router Configurations

> Remove the router configurations so that you can play other notebooks

In [ ]:
# Unconfigure the VLANs
out = nodes['L0'].execute('sudo config interface ip remove Vlan10 10.0.5.1/24')
out = nodes['L0'].execute('sudo config vlan member del 10 Ethernet2')
out = nodes['L0'].execute('sudo config vlan del 10')

out = nodes['L1'].execute('sudo config interface ip remove Vlan10 10.0.6.1/24')
out = nodes['L1'].execute('sudo config vlan member del 10 Ethernet2')
out = nodes['L1'].execute('sudo config vlan del 10')

In [ ]:
# Remove the Router configs and release the interfaces used by trex
for n in nodes:
   if (n != 'trex'):
      print("cfg cleaned")
      out = nodes[n].execute('sudo rm /etc/sonic/config_db.json')
   else:
      out = nodes[n].execute('cd /opt/cisco/trex/latest/')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py -u 00:04.0')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py -u 00:05.0')

In [50]:
!rm -rf *.json

> Do let us know of your feedback or queries about this notebook at mig-notebooks@cisco.com.